In [145]:
from tkinter import *
from random import *
from datetime import datetime
from time import *
from playsound import playsound
from IPython.display import clear_output
import os
import sys

#================================================
class Afficheur(Frame):
        code={1:(2,3),2:(1,2,7,5,4),3:(1,2,7,3,4),4:(6,7,2,3),5:(1,6,7,3,4),6:(1,6,5,4,3,7),
              7:(1,2,3),8:(1,2,3,4,5,6,7),9:(6,1,2,3,4,7),0:(1,2,3,4,5,6)}

        def __init__(self, tk, column, size=1, nCadrans=3, visible=False):
            Frame.__init__(self)
            if nCadrans < 1:
                nCadrans = 1
            self.dic={1: ( 8*size,  7*size, 23*size,  4*size), 2: (22*size,  7*size, 25*size, 19*size),
                      3: (22*size, 22*size, 25*size, 37*size), 4: (23*size, 36*size,  8*size, 39*size),
                      5: ( 5*size, 37*size,  8*size, 22*size), 6: ( 8*size, 19*size,  5*size,  7*size),
                      7: ( 8*size, 19*size, 23*size, 22*size)}
            self.configure(bg="grey40",bd=0,relief=FLAT)
            self.master.resizable(width=False, height=False)
            self.lst=[]
            self.visible=visible
            self.build(size, column, nCadrans)

        def build(self, size, c, n):
            self.can=Canvas(self,bg='black',relief=FLAT,width =25*size*n,height =42*size, bd=0, highlightthickness=0)
            self.can.grid(row=0,column=c)
            for j in range(n):
                for i in [1,2,3,4,5,6,7]:#Mise en place des segments de l'afficheur
                    liste=[]
                    v=self.dic[i]
                    liste.append(v)
                    for (w,x,y,z) in liste:
                        if(self.visible):
                            p=self.can.create_rectangle(w+(25*j),x,y+(25*j),z,fill='grey40')
                        else:
                            p=self.can.create_rectangle(w+(25*j),x,y+(25*j),z,fill='black')
                        self.lst.append(p)
            #self.initialisation()

        def quitter(self,event=None):
                self.can.delete(ALL)
                self.destroy()

        def affiche(self,arg=None):
            self.initialisation()
            if(arg != None):
                nbr_separes = [int(nbr) for nbr in str(arg) if nbr in '0123456789']
                cpt = len(self.lst)//8
                for j in range(len(nbr_separes)-1, -1, -1):
                    v=list(self.code[nbr_separes[j]])
                    for i in v:
                        self.can.itemconfig(i+(cpt*7),fill='red')
                    cpt-=1
                if(len(nbr_separes)-1 < len(self.lst)//8):
                    while cpt >= 0:
                        v=list(self.code[0])
                        for i in v:
                            self.can.itemconfig(i+(cpt*7),fill='red')
                        cpt-=1

        def initialisation(self):
            for i in self.lst:
                if(self.visible):
                    self.can.itemconfig(i,fill='grey40')
                else:
                    self.can.itemconfig(i,fill='black')
#=================================================
class Scores():

        def __init__(self, splitter="__SPLITTER__"):
            fichier = open("scores.stats", "a")
            fichier.close()
            self.splitter = splitter
        
        def write(self, pseudo, score, tailleX, tailleY, nbMines):
            fichier = open("scores.stats", "a")
            pseudo = pseudo.replace(self.splitter, "")
            if(pseudo != ""):
                fichier.write(pseudo + "__SPLITTER__" + str(score) + "__SPLITTER__" + str(tailleX) + "x" + str(tailleY) + "__SPLITTER__" + str(nbMines) + "\n")
                fichier.close()
                return True
            fichier.close()
            return False

        def read(self):
            fichier = open("scores.stats", "r")
            string = fichier.read()
            out = []
            for pseudo_score in string.split('\n'):
                liste = pseudo_score.split(self.splitter)
                if(len(liste) == 2):
                    toAppend=[self.decode(liste[0]), self.reverse(liste[1])]
                    if(toAppend[0]!=""):
                        out.append(toAppend)
            fichier.close()
            return out
#=================================================
class Demineur():

    def __init__(self, NombreColonnes=5, NombreLignes=10, ForcageNombreMines=None, Difficulte=10, OldSchool=True, DEBUG = False):
        self.__DEBUG__ = DEBUG
        self.cheat=False
        self.OLD_SCHOOL = OldSchool
        self.NB_LINES=NombreLignes
        self.NB_COLS=NombreColonnes
        self.FORCAGE_MINES=ForcageNombreMines
        self.DECALAGE=5
        self.SIDE_W=25*self.NB_COLS+self.DECALAGE*2-1
        self.SIDE_H=25*self.NB_LINES+self.DECALAGE*2-1
        self.nbMinesTrouver=0
        #difficulte de 1 à 20 (4/9/14/19/24/29/34/39/44/49/54/59/64/69/74/79/84/89/94/99% de mines)
        self.difficulte=Difficulte
        if(self.FORCAGE_MINES==None):
            self.nbMines=int((self.NB_LINES*self.NB_COLS)*((0.05*(self.difficulte%21))-0.01))
            if (self.nbMines+9) >= (self.NB_LINES*self.NB_COLS):
                self.nbMines-=9
        else:
            if((self.FORCAGE_MINES+9) >= (self.NB_LINES*self.NB_COLS)):
                self.nbMines=(self.NB_LINES*self.NB_COLS)-9
            else:
                self.nbMines=self.FORCAGE_MINES
        self.depart=True
        self.win = False
        self.perdu=False
        self.oneTime = True
        self.score_time_max = sys.maxsize * 2 + 1
        self.score_time = 0
        self.root=Tk()
        self.root.title("Démineur " + str(self.NB_COLS) + "x" + str(self.NB_LINES))
        #Afficheur Temps:
        if self.OLD_SCHOOL:
            self.temps=Afficheur(self.root, column=0)
            self.temps.grid(row=0, column=0)
            self.temps.affiche(0)
        else:
            self.temps = Label(text="00:00:00", font=('Helvetica', 24), fg='black')
            self.temps.grid(row=0, column=0)
        #Afficheur Mines:
        if self.OLD_SCHOOL:
            self.mines=Afficheur(self.root, column=2)
            self.mines.grid(row=0, column=2)
            self.mines.affiche(0)
        else:
            self.mines = Label(text=str(self.nbMinesTrouver)+"/"+str(self.nbMines), font=('Helvetica', 24), fg='black')
            self.mines.grid(row=0, column=2)
        #Plateau du demineur:
        self.cnv=Canvas(self.root, width=self.SIDE_W, height=self.SIDE_H, bg="white")
        self.cnv.bind("<Button-1>", self.ClicGauche)
        self.cnv.bind("<Button-3>", self.ClicDroit)
        self.root.bind("<Control-Shift-KeyPress-C>", self.LeCheatInfernal)
        self.cnv.grid(row=1, column=1)
        #Score:
        self.scores = Scores()
        #Chargement des images:
        self.mine = PhotoImage(file="img/mine.gif")
        self.mine_explosee = PhotoImage(file="img/mine_explosee.gif")
        self.drapeau = PhotoImage(file="img/drapeau.gif")
        self.grille=self.generationGrilleVide()
        self.refreshScreen()
        self.root.mainloop()
    
    def LeCheatInfernal(self, event):
        if self.depart==True:
            print("CheatInfernal activé! Aucunes mines!")
            self.cheat=True
            self.root.title("Démineur " + str(self.NB_COLS) + "x" + str(self.NB_LINES) + " [CheatInfernal]")
            self.nbMines=0

    #Indiquer la case par un drapeau/un point d'interrogation/rien:
    def ClicDroit(self, event):
        if not(self.win) and not(self.perdu):
            # position du pointeur de la souris
            X = event.x
            Y = event.y
            col, line = self.getCase(X, Y)
            if (self.NB_LINES > line >= 0) and (self.NB_COLS > col >=0):
                #Debug:
                if self.__DEBUG__ :
                    print(X, Y)
                    print(col, line)
                #Rafraichissement de la case choisit:
                if col<len(self.grille[1]) and line<len(self.grille[1][0]):
                    self.changeCase(col, line)
                #Rafraichissement du jeu:
                self.refreshScreen()
                #Debug:
                if self.__DEBUG__ :
                    # on dessine un carré
                    r = 2
                    self.cnv.create_rectangle(X-r, Y-r, X+r, Y+r, outline="black",fill="red")

    #Révéler une case (ou plusieurs):
    def ClicGauche(self, event):
        if not(self.win) and not(self.perdu):
            # position du pointeur de la souris
            X = event.x
            Y = event.y
            col, line = self.getCase(X, Y)
            if self.__DEBUG__ :
                print(X, Y)
                print(col, line)
            if (self.NB_LINES > line >= 0) and (self.NB_COLS > col >=0):
                #Mecanique premier tour:
                if self.depart==True:
                    self.firstTour(col, line)
                #Mecanique grille du jeu:
                if col<len(self.grille[1]) and line<len(self.grille[1][0]):
                    self.revelerCase(col, line)
                #Rafraichissement du jeu:
                self.refreshScreen()
                #Debug:
                if self.__DEBUG__ :
                    # on dessine un carré
                    r = 2
                    self.cnv.create_rectangle(X-r, Y-r, X+r, Y+r, outline="black",fill="green")

    def revelerCase(self, x, y):
        if self.grille[1][x][y] == 0:
            if self.grille[0][x][y]:
                self.grille[1][x][y]=4
                self.revelerGrille(x, y)
                self.perdu = True
            else:
                self.grille[1][x][y]=3
                if self.nbMinesAutour(x, y)==0:
                    listeCasesVides = self.detectCasesVidesAutour(x, y, [])
                    while(len(listeCasesVides) > 0):
                        caseVide = listeCasesVides[0]
                        listeCasesVides.remove(caseVide)
                        listeCasesVides = self.detectCasesVidesAutour(caseVide[0], caseVide[1], listeCasesVides)

    def detectCasesVidesAutour(self, x, y, casesVides):
        V = self.voisins(x, y)
        for case in V:
            if self.grille[0][case[0]][case[1]] == False:
                if (self.nbMinesAutour(case[0], case[1]) == 0) and not(self.grille[1][case[0]][case[1]] == 3):
                    casesVides.append(case)
                self.grille[1][case[0]][case[1]]=3
        return casesVides

    def nbMinesAutour(self, x, y):
        V = self.voisins(x, y)
        nbMines=0
        for case in V:
            if self.grille[0][case[0]][case[1]]:
                nbMines+=1
        return nbMines

    #Retourne les cases autours:
    def voisins(self, i, j):
        return [(a, b) for (a, b) in [(i+1, j+1), (i, j+1), (i-1, j+1), (i+1, j), (i-1, j), (i+1, j-1), (i, j-1), (i-1, j-1)]
                if a in range(self.NB_COLS) and b in range(self.NB_LINES)]

    def firstTour(self, X, Y):
        self.grille = self.generationGrille(self.nbMines, X, Y)
        self.start_time = datetime.now().timestamp()
        self.actualiserTimer()
        self.depart=False

    def changeCase(self, y, x):
        if 2 >= self.grille[1][y][x] >= 0:
            self.grille[1][y][x]=(self.grille[1][y][x]+1)%3
        return self.grille[1][y][x]

    def getCase(self, x, y):
        return (x-self.DECALAGE)//25, (y-self.DECALAGE)//25

    #fonction revelant les mines si on perd la partie
    def revelerGrille(self, y, x):
        playsound("sounds/explosion.mp3")
        for i in range(len(self.grille[1])):
            for j in range(len(self.grille[1][0])):
                if not(i==y and j==x) and self.grille[0][i][j]:
                    self.grille[1][i][j]=3

    #fonction revelant les mines si on perd la partie
    def nbMinesDecouvertes(self):
        nbMines=0
        for i in range(len(self.grille[1])):
            for j in range(len(self.grille[1][0])):
                if self.grille[1][i][j]==1:
                    nbMines+=1
        return nbMines

    def generationGrilleVide(self):
        return [[[False for j in range(self.NB_LINES)] for i in range(self.NB_COLS)],
                [[0 for j in range(self.NB_LINES)] for i in range(self.NB_COLS)]]

    def generationGrille(self, nbMines, X, Y):
        G=[]
        Gmines=[]
        for i in range(self.NB_COLS):
            subGrille=[]
            for j in range(self.NB_LINES):
                subGrille.append(False)
            Gmines.append(subGrille)
        G.append(Gmines)
        Gaff=[]
        for i in range(self.NB_COLS):
            subGrille=[]
            for j in range(self.NB_LINES):
                subGrille.append(0)
            Gaff.append(subGrille)
        G.append(Gaff)
        whiteList = self.voisins(X, Y)
        whiteList.append((X, Y))
        while(nbMines>0):
            x=randint(0, self.NB_COLS-1)
            y=randint(0, self.NB_LINES-1)
            if not( (x, y) in whiteList):
                if not(G[0][x][y]):
                    G[0][x][y]=True
                    nbMines-=1
        return G

    def refreshScreen(self):
        #Suppression des images et rectangles dessines 
        self.cnv.delete("all")
        #Placement des images
        for line in range(self.NB_LINES):
            for col in range(self.NB_COLS):
                self.cnv.create_rectangle(self.DECALAGE+col*25, self.DECALAGE+line*25, self.DECALAGE+col*25+25, self.DECALAGE+line*25+25, fill='lightgray')
                if self.grille[1][col][line]==1:
                    self.centre=(self.DECALAGE+13+col*25, self.DECALAGE+13+line*25)
                    self.cnv.create_image(self.centre, image=self.drapeau)
                elif self.grille[1][col][line]==2:
                    self.cnv.create_text(self.DECALAGE*3+self.DECALAGE//2+col*25, self.DECALAGE*3+self.DECALAGE//2+line*25,fill="yellow",font="Helvetica 10 bold", text="?")
                elif self.grille[1][col][line]==3 or self.grille[1][col][line]==4:
                    if self.grille[0][col][line]:
                        self.centre=(self.DECALAGE+13+col*25, self.DECALAGE+13+line*25)
                        if self.grille[1][col][line]==4:
                            self.cnv.create_image(self.centre, image=self.mine_explosee)
                        else:
                            self.cnv.create_image(self.centre, image=self.mine)
                    else:
                        self.nbMinesArround = self.nbMinesAutour(col, line)
                        if not(self.nbMinesArround==0):
                            self.cnv.create_text(self.DECALAGE*3+self.DECALAGE//2+col*25, self.DECALAGE*3+self.DECALAGE//2+line*25,fill="darkblue",font="Helvetica 10 bold", text=str(self.nbMinesArround))
                else:
                    self.cnv.create_rectangle(self.DECALAGE+col*25+1, self.DECALAGE+line*25+1, self.DECALAGE+col*25+23, self.DECALAGE+line*25+23, fill='gray')
        #Debug:
        if self.__DEBUG__ :
            #Clear Windows:
            os.system('cls')
            #Clear Linux:
            os.system('clear')
            #Clear Jupyter:
            clear_output(wait=True)
            print("Nombre de mines dans le niveau:", self.nbMines, "pour", self.NB_LINES*self.NB_COLS, "cases.")
            print("Type numéro grille:")
            print("0 --> Case vide non-découverte")
            print("1 --> Case vide avec drapeau")
            print("2 --> Case vide avec '?'")
            print("3 --> Case découverte")
            print()
            print("M --> Case avec mine")
            for line in range(self.NB_LINES):
                for col in range(self.NB_COLS):
                    print("--------", end="")
                print("")
                print("|", end="")
                for col in range(self.NB_COLS):
                    if self.grille[0][col][line]:
                        print(self.grille[1][col][line], "(M)", end="\t")
                    else:
                        print(self.grille[1][col][line], "   ", end="\t")
                print("|")
            for col in range(self.NB_COLS):
                print("--------", end="")
            print("")

    def checkWin(self):
        self.nbMinesTrouver=0
        nbMinesAvecDrapeaux=0
        casesToutesDecouvertes = True
        for line in range(self.NB_LINES):
            for col in range(self.NB_COLS):
                if self.grille[1][col][line]==0:
                    casesToutesDecouvertes = False
                if self.grille[1][col][line]==1:
                    self.nbMinesTrouver+=1
                    if self.grille[0][col][line]:
                        nbMinesAvecDrapeaux+=1
        return (nbMinesAvecDrapeaux == self.nbMines) and casesToutesDecouvertes

    def actualiserTimer(self):
        self.win = self.checkWin()
        if self.OLD_SCHOOL:
            self.mines.affiche(self.nbMinesTrouver)
        else:
            self.mines.configure(text=str(self.nbMinesTrouver)+"/"+str(self.nbMines))
        if not(self.win) and not(self.perdu):
            self.score = int(time() - self.start_time)
            if self.OLD_SCHOOL:
                self.temps.affiche(self.score)
            else:
                self.temps.configure(text=strftime("%H:%M:%S", gmtime(self.score)))
        elif self.win and self.oneTime:
            #playsound("sounds/ta_da.mp3")
            print("Partie gagné !")
            self.scoreboard=Tk()
            self.scoreboard.title("Scoreboard")
            self.labelPseudo = Label(self.scoreboard, text='Entrez votre pseudo :')
            self.labelPseudo.grid(row=0, column=0)
            self.entryPseudo = Entry(self.scoreboard, textvariable=StringVar())
            self.entryPseudo.grid(row=1, column=0)
            self.buttonPseudo = Button(self.scoreboard, text='Ok', command=lambda:self.addScore(self.entryPseudo.get()))
            self.buttonPseudo.grid(row=1, column=1)
            self.scoreboard.mainloop()
            self.oneTime = False
        self.root.after(500, self.actualiserTimer)

    def addScore(self, pseudo):
        #Debug:
        if self.__DEBUG__ :
            print("Cheat ?", self.cheat)
            print("Pseudo:", pseudo)
            print("Score:", self.score)
            print("Nombre de mines:", self.nbMines)
            print("Taille grille (X Y):", self.NB_COLS, self.NB_LINES)
        #ajouter le score à la liste:
        if self.cheat:
            self.scores.write(pseudo + " cheateur", self.score, self.NB_COLS, self.NB_LINES, self.nbMines)
        else:
            self.scores.write(pseudo, self.score, self.NB_COLS, self.NB_LINES, self.nbMines)
        self.labelPseudo.destroy()
        self.entryPseudo.destroy()
        self.buttonPseudo.destroy()
        self.scoreboard.destroy()
#=================================================


#===============Programme Principal===============
#Demineur(OldSchool=True)
Demineur(NombreColonnes=5, NombreLignes=10, ForcageNombreMines=None, Difficulte=2, OldSchool=True, DEBUG = True)
#=================================================

Nombre de mines dans le niveau: 0 pour 50 cases.
Type numéro grille:
0 --> Case vide non-découverte
1 --> Case vide avec drapeau
2 --> Case vide avec '?'
3 --> Case découverte

M --> Case avec mine
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
|3    	3    	3    	3    	3    	|
----------------------------------------
Partie gagné !
Cheat 